# 네이버 지식인 크롤링

In [27]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
from urllib.parse import quote

In [28]:
chromedriver = '../05.Crawling/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

In [29]:
base_url = 'https://kin.naver.com/search/list.naver?query='
url = base_url + quote('여친선물')
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [30]:
lis = soup.select('ul.basic1 > li')
len(lis)

10

In [31]:
li = lis[0]
sub_url = li.select_one('a._searchListTitleAnchor')['href']
sub_url

'https://kin.naver.com/qna/detail.naver?d1id=8&dirId=80101&docId=395035694&qb=7Jes7Lmc7ISg66y8&enc=utf8&section=kin&rank=1&search_sort=0&spq=0'

In [32]:
driver.get(sub_url)
time.sleep(2)

In [33]:
count = int(driver.find_element_by_css_selector('._answerCount.num').text)
answer_page = (count - 1) // 5
print(count, answer_page)

8 1


In [34]:
for i in range(answer_page):
    driver.find_element_by_id('nextPageButton').click()
    time.sleep(2)

In [35]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
divs = soup.select('div.answer-content__item._contentWrap._answer')
len(divs)

8

In [36]:
div = divs[0]
ps = div.select('p.se-text-paragraph')
for p in ps:
    print(p.get_text())

혹시 여친분이 생리통이 있으시다면..여친선물로 복부온열기 어떠세요?
저 생리통 너무 심해서 이번에 남자친구가 복부온열기 선물줬는데..
뭔가 감동이더라구요. 절 많이 생각해서 준 선물 같이 느껴지더라구요.
하고있으면 배가 뜨듯하고 진동마사지도 해줘서 생리통 거의 안느껴지고 좋아요!!
​
제가 쓰는건 배뜨끈 복부온열기이예요. 아래에 링크알려드릴게요.
좋은 여친선물이 되셨으면 좋겠네요.


In [37]:
divs[6].select

<bound method Tag.select of <div class="answer-content__item _contentWrap _answer" id="answer_3">
<a href="#answer3" name="answer3"><span class="blind">3번째 답변</span></a>
<div class="c-heading-answer">
<div class="c-heading-answer__profile">
<div class="profile-default">
<span class="profile-default--inner">
<span class="profile-default__thumbnail" style="background-image: url(https://ssl.pstatic.net/static/kin/09renewal/avatar/200x200_m_gray/answer.png);"><span class="blind">삭제 답변 프로필 이미지</span></span>
</span>
</div>
</div>
<div class="c-heading-answer__body">
<div class="c-heading-answer__title">
<p class="title">
				
					
						작성자가 직접 삭제한 답변입니다.
					
				
			</p>
<div class="c-userinfo">
<span class="c-userinfo__item">삭제일 <em>2021.07.19.</em></span>
</div>
</div>
</div>
</div>
</div>>

In [38]:
driver.back()
time.sleep(2)

In [39]:
answer_list = []
for page in tqdm(range(1, 11)):
    url = f'{base_url}{quote("여친선물")}&page={page}'
    driver.get(url)
    time.sleep(2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    lis = soup.select('ul.basic1 > li')
    for li in lis:
        sub_url = li.select_one('a._searchListTitleAnchor')['href']
        driver.get(sub_url)
        time.sleep(2)

        count = int(driver.find_element_by_css_selector('._answerCount.num').text)
        answer_page = (count - 1) // 5
        for i in range(answer_page):
            driver.find_element_by_id('nextPageButton').click()
            time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        divs = soup.select('div.answer-content__item._contentWrap._answer')
        print(len(divs))
        for div in divs:
            ps = div.select('p.se-text-paragraph')
            paragraph = ''
            for p in ps:
                text = p.get_text()
                if text == ' ' or text == '\u200b':
                    continue
                paragraph += text + '\n'
            answer_list.append(paragraph)
        driver.back()
        time.sleep(2)

  0%|          | 0/10 [00:00<?, ?it/s]

8
10
7
12
14
9
1
4
1
8
3
7
15
9
10
2
12
15
5
12
2
1
5
13
4
1
5
5
14
8
1
12
5
13
5
1
2
1
20
13
11
11
7
9
1
1
5
1
4
18
3
5
8
2
1
6
12
1
11
1
15
5
7
1
4
16
6
14
16
2
7
2
11
6
1
7
9
4
8
21
3
4
3
11
1
6
1
6
1
2
1
7
1
5
6
2
8
1
9
26


In [40]:
driver.close()

In [41]:
len(answer_list)

679

In [42]:
answer_list[0]

'혹시 여친분이 생리통이 있으시다면..여친선물로 복부온열기 어떠세요?\n저 생리통 너무 심해서 이번에 남자친구가 복부온열기 선물줬는데..\n뭔가 감동이더라구요. 절 많이 생각해서 준 선물 같이 느껴지더라구요.\n하고있으면 배가 뜨듯하고 진동마사지도 해줘서 생리통 거의 안느껴지고 좋아요!!\n제가 쓰는건 배뜨끈 복부온열기이예요. 아래에 링크알려드릴게요.\n좋은 여친선물이 되셨으면 좋겠네요.\n'

In [43]:
with open('data/new_gift.txt', 'w', encoding='utf-8') as fp:
    for answer in answer_list:
        fp.write(answer+'\n')